# Processing WTCCC Summary Statistics Data
The following data was downloaded from EBI-EGA accessions EGAD00000000002-EGAD00000000009
Access to data was requested first from https://www.wtccc.org.uk/.
The data was decrypted with a key generated by the EGA helpdesk and using the EGA download client.

The p-values reported in the paper come from the "basic_snptest" files. Each numbered file for each disease is the SNP associations for all SNPs found on that chromosome. Each disease has a 2 or 3 letter disease code. These tables will all be aggregated together into a single table reporting the SNP results. The p-value reported will be the smaller of the 2 values in the 'frequentist_add' or 'frequentist_gen' columns of these files.

In [2]:
import os
import pandas as pd

In [3]:
wd = '/cellar/users/jkhuang/WTCCC_Download/WTCCC_summary_stats/'
outdir = '/cellar/users/jkhuang/Data/Projects/Network_GWAS/Data/WTCCC/WTCCC_SNPs/'

In [4]:
# Get all the files with SNP summary statistics
snp_files = [fn for fn in os.listdir(wd) if '_basic_snptest_' in fn]
snp_files.sort()

In [5]:
# Get all the disease codes from the file names
diseases = []
for fn in snp_files:
    disease_code = fn.split('_')[6]
    if disease_code not in diseases:
        diseases.append(disease_code)
    else:
        pass
diseases.sort()
print 'Diseases:', diseases

Diseases: ['BD', 'CAD', 'CD', 'HT', 'RA', 'T1D', 'T2D']


In [49]:
# Construct concatenated SNP table and save to file
for disease in diseases:
    disease_snp_files = [fn for fn in snp_files if disease in fn]
    disease_chr_snp_tables = []
    for fn in disease_snp_files:
        disease_chr_snp_table = pd.read_csv(wd+fn, sep='\t')
        chrom = int(fn.split('_')[-1].split('.')[0])
        disease_chr_snp_table['Chromosome'] = chrom
        disease_chr_snp_table['P-Value'] = disease_chr_snp_table[['frequentist_add', 'frequentist_gen']].min(axis=1)
        disease_chr_snp_table_filt = disease_chr_snp_table[['rsid', 'Chromosome', 'pos', 'frequentist_add', 'frequentist_gen', 'P-Value']]
        disease_chr_snp_tables.append(disease_chr_snp_table_filt)
    disease_full_snp_table = pd.concat(disease_chr_snp_tables).reset_index(drop=True)
    disease_full_snp_table.columns = ['rsID', 'Chromosome', 'Pos', 'Trend P-Value', 'Genotypic P-Value', 'Min P-Value']
    disease_full_snp_table_filt = disease_full_snp_table[disease_full_snp_table['rsID']!='---'].reset_index(drop=True)
    disease_full_snp_table_filt.to_csv(outdir+disease+'_full_WTCCC_snptest_summary_table.csv', index=False, sep='\t')

In [6]:
# Generate bed files to be used for LiftOver for WTCCC data
from nbgwas import NBGWAS_snp2gene as snp2gene
WTCCC_data_dir = outdir
for fn in os.listdir(WTCCC_data_dir):
    if fn.endswith('_summary_table.csv'):
        disease = fn.split('_')[0]
        print 'Disease:', disease
        WTCCC_snp_table = snp2gene.load_SNP_pvals(WTCCC_data_dir+fn, delimiter='\t', header=True, cols='0,1,2,5')
        WTCCC_snp_table.columns = ['name', 'chrom_num', 'chromEnd', 'pvalue']
        WTCCC_snp_table['chromStart'] = WTCCC_snp_table['chromEnd']-1
        WTCCC_snp_table['chrom'] = ['chr'+repr(c) for c in WTCCC_snp_table['chrom_num']]
        WTCCC_snp_bed = WTCCC_snp_table[['chrom', 'chromStart', 'chromEnd', 'name']]
        WTCCC_snp_bed.to_csv(WTCCC_data_dir+fn[:-4]+'.bed', sep='\t', header=False, index=False)

Disease: T1D
Disease: T2D
Disease: HT
Disease: BD
Disease: CD
Disease: RA
Disease: CAD


In [42]:
# After liftover, re-merge data
for disease in diseases:
    hg19_pos = pd.read_csv(WTCCC_data_dir+disease+'_WTCCC_snplocs_galaxy-LiftOver-hg19.bed', sep='\t', header=-1)
    hg19_pos.columns = ['chrom', 'chromStart', 'chromEnd', 'name']
    hg19_pos = hg19_pos.set_index('name')
    WTCCC_snp_table = pd.read_csv(WTCCC_data_dir+disease+'_full_WTCCC_snptest_summary_table.csv',
                                  delimiter='\t').set_index('rsID')
    hg19_WTCCC_snp_table = pd.concat([hg19_pos, WTCCC_snp_table], axis=1).dropna()
    hg19_WTCCC_snp_table_filt = hg19_WTCCC_snp_table[['Chromosome', 'chromEnd', 'Pos', 'Trend P-Value', 'Genotypic P-Value', 'Min P-Value']].reset_index(drop=False)
    hg19_WTCCC_snp_table_filt.columns = ['Marker', 'Chromosome', 'hg19 Pos', 'NCBI35 Pos', 'Trend P-Value', 'Genotypic P-Value', 'Min P-Value']
    hg19_WTCCC_snp_table_filt['hg19 Pos'] = hg19_WTCCC_snp_table_filt['hg19 Pos'].astype(int)
    hg19_WTCCC_snp_table_sorted = hg19_WTCCC_snp_table_filt.sort_values(by=['Min P-Value', 'Chromosome', 'hg19 Pos'])
    hg19_WTCCC_snp_table_sorted.to_csv(WTCCC_data_dir+disease+'_WTCCC_snptest_galaxy-LiftOver-hg19_summary_table.csv', sep='\t', index=False)

In [43]:
hg19_WTCCC_snp_table_sorted.head()

,Marker,Chromosome,hg19 Pos,NCBI35 Pos,Trend P-Value,Genotypic P-Value,Min P-Value
179987,rs17480050,8,19381917,19426197,1.754650e-214,1.000000e+00,1.754650e-214
270213,rs3777582,6,45914028,46022006,1.844440e-20,5.026180e-121,5.026180e-121
46341,rs11042656,11,10214455,10171031,6.606760e-18,6.172960e-111,6.172960e-111
66823,rs11705626,22,25435959,23760513,7.880970e-76,1.399170e-93,1.399170e-93
121222,rs1477523,7,136805138,136262393,3.707340e-06,5.104710e-75,5.104710e-75


# Constructing NCBI Build 35 gene locations map
Constructing this map requires the following files:

NCBI Build 35 Known Gene Locations: http://hgdownload.soe.ucsc.edu/goldenPath/hg17/database/knownGene.txt.gz  
Known Gene Alias Map: http://hgdownload.soe.ucsc.edu/goldenPath/hg17/database/kgXref.txt.gz  
Current HUGO Gene Symbols Table: [ftp://ftp.ebi.ac.uk/pub/databases/genenames/new/tsv/locus_types/gene_with_protein_product.txt](ftp://ftp.ebi.ac.uk/pub/databases/genenames/new/tsv/locus_types/gene_with_protein_product.txt)


In [50]:
wd2 = '/cellar/users/jkhuang/Data/Projects/Network_GWAS/Data/WTCCC/NCBI_Build35/'

In [55]:
# Load known gene locations
knownGenes = pd.read_csv(wd2+'knownGene.txt', sep='\t', header=-1)
knownGenes.columns = ['Name', 'Chr', 'Strand', 'txStart', 'txEnd', 'cdsStart', 'cdsEnd', 'exonCount', 'exonStart', 'exonEnd', 'UniprotID', 'GencodeID']

In [463]:
# Keep only known gene locations on autosomes
autosomes = ['chr'+str(i) for i in range(1,23)]
autosome_knownGenes = knownGenes[knownGenes['Chr'].isin(autosomes)]
autosome_knownGenes = autosome_knownGenes[['Name', 'Chr', 'Strand', 'txStart', 'txEnd']]
autosome_knownGenes = autosome_knownGenes.reset_index(drop=True)
# Remove duplicates after only keeping chromosome, strand, and location information
autosome_knownGenes_filt = autosome_knownGenes.set_index('Name').drop_duplicates().reset_index(drop=False)

In [465]:
knownGene_names = list(set(autosome_knownGenes_filt['Name']))

In [63]:
# Load known gene Aliases
aliases = pd.read_csv(wd2+'kgXref.txt', sep='\t', header=-1)
aliases.columns=['Gene', 'mRNAID', 'UniprotAccession', 'UniprotID', 'Gene Symbol', 'RefSeq ID', 'Entrez', 'Description']

In [155]:
# Load HGNC gene symbol table
HGNC_table = pd.read_csv(wd2+'HGNC_Gene_Table_2018-03-20.txt', sep='\t')

# Set all valid HGNC symbols
HGNC_symbols = list(HGNC_table['symbol'])

In [497]:
# Map all known HGNC aliases
HGNC_alias_map = {}
for i in HGNC_table.index:
    symbol = HGNC_table.ix[i]['symbol']
    HGNC_alias_map[symbol]=[symbol]
    
    # Map gene symbol aliases
    if type(HGNC_table.ix[i]['alias_symbol'])==str:
        alias_symbols = [name.upper() for name in HGNC_table.ix[i]['alias_symbol'].split('|')]
        for alias_symbol in alias_symbols:
            if alias_symbol not in HGNC_alias_map:
                HGNC_alias_map[alias_symbol]=[symbol]
            else:
                if symbol not in HGNC_alias_map[alias_symbol]:
                    HGNC_alias_map[alias_symbol].append(symbol)
    else:
        pass
    
    # Map previous gene symbol names
    if type(HGNC_table.ix[i]['prev_symbol'])==str:
        prev_symbol = HGNC_table.ix[i]['prev_symbol']
        if prev_symbol not in HGNC_alias_map:
            HGNC_alias_map[prev_symbol] = [symbol]
        else:
            HGNC_alias_map[alias_symbol].append(symbol)

In [534]:
# Clean up HGNC aliases to point at only itself
for gene in HGNC_alias_map:
    if (gene in HGNC_alias_map[gene]) & (len(HGNC_alias_map[gene]) > 1):
        HGNC_alias_map[gene] = [gene]

In [568]:
multiple_chromosomes = []
failed_mappings = []
geneloc_mapping = []
multiple_mappings = []
for gene in knownGene_names:
    genelocs = autosome_knownGenes_filt[autosome_knownGenes_filt['Name']==gene]
    # Get gene locations
    if len(set(genelocs['Chr']))==1:
        chrom = int(list(set(genelocs['Chr']))[0][3:])
        TxStarts, TxEnds = [], []
        for i in genelocs.index:
            if genelocs.ix[i]['Strand']=='+':
                TxStarts.append(genelocs.ix[i]['txStart'])
                TxEnds.append(genelocs.ix[i]['txEnd'])
            else:
                TxStarts.append(genelocs.ix[i]['txEnd'])
                TxEnds.append(genelocs.ix[i]['txStart'])
        TxStart, TxEnd = min(TxStarts), max(TxEnds)
    else:
        multiple_chromosomes.append(gene)
        continue
    
    # Get all aliases
    gene_aliases = aliases[aliases['Gene']==gene]
    gene_symbols = list(gene_aliases['Gene Symbol'].dropna())
    # Map all alias symbols to official symbol
    official_symbols = []
    for symbol in gene_symbols:
        if symbol in HGNC_alias_map:
            official_symbols = official_symbols + HGNC_alias_map[symbol]
        else:
            official_symbols.append(symbol)
    HGNC_genes_1 = HGNC_table[HGNC_table['symbol'].isin(list(set(official_symbols)))]
    # Map any refseq ids to HGNC_table
    refseq_ids = list(gene_aliases['RefSeq ID'].dropna())
    HGNC_genes_2 = HGNC_table[HGNC_table['refseq_accession'].isin(list(set(refseq_ids)))]
    # Map any uniprot ids to HGNC_table
    uniprot_ids = list(gene_aliases['UniprotAccession'].dropna()) + list(gene_aliases['UniprotID'].dropna())
    HGNC_genes_3 = HGNC_table[HGNC_table['uniprot_ids'].isin(list(set(uniprot_ids)))]
    
    # Concatenate all results together
    HGNC_genes = pd.concat([HGNC_genes_1, HGNC_genes_2, HGNC_genes_3]).drop_duplicates()
    if HGNC_genes.shape[0]==0:
        failed_mappings.append(gene)
        continue
    elif HGNC_genes.shape[0]==1:
        gene_symbol = list(HGNC_genes['symbol'])[0]
        geneloc_mapping.append([gene, gene_symbol, chrom, TxStart, TxEnd])
    else:
        print gene
        if gene.startswith('NM_'):
            mapped_symbol_list = list(HGNC_genes[HGNC_genes['refseq_accession']==gene]['symbol'])
            if len(mapped_symbol_list)==1:
                gene_symbol = mapped_symbol_list[0]
                geneloc_mapping.append([gene, gene_symbol, chrom, TxStart, TxEnd])
            else:
                multiple_mappings.append([gene, HGNC_genes[['symbol', 'alias_symbol', 'prev_symbol', 'uniprot_ids', 'refseq_accession']]])
        else:
            mapped_symbol_list = list(HGNC_genes[HGNC_genes['uniprot_ids']==gene]['symbol'])
            if len(mapped_symbol_list)==1:
                gene_symbol = mapped_symbol_list[0]
                geneloc_mapping.append([gene, gene_symbol, chrom, TxStart, TxEnd])
            else:
                multiple_mappings.append([gene, HGNC_genes[['symbol', 'alias_symbol', 'prev_symbol', 'uniprot_ids', 'refseq_accession']]])

NM_001010890
NM_001004060
NM_001004067
NM_001786
NM_173490
NM_001003702
NM_032308
NM_006876
NM_005324
NM_003910
NM_181538
NM_015576
NM_033546
NM_020640
NM_178449
NM_178445
NM_003616
NM_002107
NM_080820
NM_001005287
BT007314
NM_018006
NM_152250
NM_152251
NM_030663
NM_002567
NM_033377
NM_001001317
NM_014226
NM_152499
NM_016283
NM_181809
NM_001004697
NM_145047
NM_018556
NM_016532
NM_000204
CR608316
NM_182831
NM_000398
NM_144565
NM_001091
NM_017979
NM_014727
NM_175054
NM_006250
NM_015089
NM_024311
NM_177926
NM_003299
NM_001004136
NM_022452
NM_021182
AK021680
NM_003482
BC001926
NM_207355
NM_006067
NM_004038
NM_018671
NM_024888
NM_001006109
AB023508
BC061588
NM_020755
AB037703
NM_001009610
NM_006460
AK223365
NM_020536
NM_024021
NM_003537
NM_003531
NM_001001786
NM_032495
BC018650
NM_198428
NM_003536
NM_003534
NM_003535
NM_003532
NM_003533
NM_003530
NM_003538
NM_003539
NM_002665
NM_000344
NM_057159
NM_176870
NM_004942
AK098544
NM_004084
NM_031950
NM_000558
NM_033542
NM_032038
NM_012292
NM_0010

In [584]:
mapped_symbols = list(set(NCBI_Build35_genelocs['Symbol']))
NCBI_Build35_genelocs_collapsed = []
for gene in mapped_symbols:
    if len(set(NCBI_Build35_genelocs[NCBI_Build35_genelocs['Symbol']==gene]['Chromosome'])) == 1:
        genelocs = NCBI_Build35_genelocs[NCBI_Build35_genelocs['Symbol']==gene]
        chrom = list(set(genelocs['Chromosome']))[0]
        txStart = genelocs['TxStart'].min()
        txEnd = genelocs['TxEnd'].max()
        NCBI_Build35_genelocs_collapsed.append([gene, chrom, txStart, txEnd])
NCBI_Build35_genelocs_table = pd.DataFrame(NCBI_Build35_genelocs_collapsed, columns=['Gene', 'Chromosome', 'TxStart', 'TxEnd'])        

In [588]:
NCBI_Build35_genelocs_table = NCBI_Build35_genelocs_table.sort_values(by=['Chromosome', 'TxStart'], ascending=[True, True])
NCBI_Build35_genelocs_table.to_csv('/cellar/users/jkhuang/Data/Projects/Network_GWAS/Data/WTCCC/NCBI_Build35/NCBI_Build35_HGNC_genelocs.csv', 
                                   sep='\t', index=False)